<a href="https://colab.research.google.com/github/sean-halpin/chess_website/blob/models_init/models/Chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Lichess Puzzles

In [ ]:
!pip install zstandard

In [3]:
import requests
import zstandard
import csv
import io

url = "https://database.lichess.org/lichess_db_puzzle.csv.zst"
compressed_file_path = "lichess_db_puzzle.csv.zst"
output_csv_path = "lichess_db_puzzle.csv"

# Download the Zstandard compressed file
response = requests.get(url)
if response.status_code == 200:
    with open(compressed_file_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully to {compressed_file_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

# Extract the Zstandard compressed file to CSV
with open(compressed_file_path, 'rb') as compressed_file:
    dctx = zstandard.ZstdDecompressor()
    with dctx.stream_reader(compressed_file) as reader:
        with io.TextIOWrapper(reader, encoding='utf-8') as text_reader:
            with open(output_csv_path, 'w', newline='', encoding='utf-8') as output_csv:
                writer = csv.writer(output_csv)

                for line in text_reader:
                    decoded_line = line.strip()
                    csv_row = decoded_line.split(',')
                    writer.writerow(csv_row)

print(f"File extracted successfully to {output_csv_path}")


File downloaded successfully to lichess_db_puzzle.csv.zst
File extracted successfully to lichess_db_puzzle.csv


# Load Puzzle Data

In [ ]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = "lichess_db_puzzle.csv"

# Use read_csv to load the data into a DataFrame
df = pd.read_csv(csv_file_path, sep=",")

# df_description = df.describe()

# # Print the summary
# print(df_description)

# Explore Data

In [ ]:
# Display the first few rows of the DataFrame to verify the data has been loaded
pd.set_option('display.max_rows', None)
print(df.dtypes)
df.head(100)

In [ ]:
print(df.dtypes)
df.describe()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
print(df.iloc[84]['FEN'])
print(df.iloc[84]['Moves'])

In [ ]:
# Select only the FEN and moves columns
selected_columns = ["FEN", "Moves"]
df_subset = df[selected_columns]

In [ ]:
df_subset.head()

In [ ]:
df_subset.iloc[84]

In [ ]:
!pip install python-chess

In [ ]:
def move_to_index(move):
    """
    Convert chess move (e.g., "e2e4") to index number based on the specified mapping.
    """
    file_mapping = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    rank_mapping = {str(i+1): i * 8 for i in range(8)}

    start_square = move[:2]
    end_square = move[2:]

    start_index = file_mapping[start_square[0]] + rank_mapping[start_square[1]]
    end_index = file_mapping[end_square[0]] + rank_mapping[end_square[1]]

    return start_index, end_index

In [ ]:
import chess

# Iterate through each row in the DataFrame
for index, row in df_subset.head(1).iterrows():
    # Get the FEN string from the current row
    fen_string = row["FEN"]

    # Create a chess.Board object from the FEN string
    board = chess.Board(fen_string)

    for move in row["Moves"].split(" "):
      # Print the board state and moves
      print(move, "\n")
      print(f"Board state for puzzle {index + 1} (FEN: {board.board_fen()}):")
      print(board)
      print("Moves:", row["Moves"])
      print("\n" + "=" * 30 + "\n")
      # Extract and print the piece at each square
      i=0
      for square in chess.SQUARES:
          piece = board.piece_at(square)
          print(f"{i} - Square {chess.square_name(square)}: {piece}")
          i+=1
      print(move_to_index(move))
      m = chess.Move.from_uci(move)
      board.push(m)  # Make the move
      break


In [ ]:
board

In [ ]:
print(dir(board.piece_at(15)))
print(board.piece_at(15).piece_type)
print(board.piece_at(22).color)
print(board.piece_at(22).symbol())

In [ ]:
print(board.piece_at(44).__str__())
print(board.piece_at(0).__str__())

# Prepare Data

In [ ]:
sample_count = 50000
sampled_df = df_subset.sample(n=50000, random_state=42)

In [ ]:
# Create an empty list to store row data
rows_data = []

i=0
# Iterate through each row in the DataFrame
for index, row in sampled_df.iterrows():
    # Get the FEN string from the current row
    fen_string = row["FEN"]

    # Create a chess.Board object from the FEN string
    try:
        board = chess.Board(fen_string)
    except:
        print("An exception occurred")
        continue

    for move in row["Moves"].split(" "):
        try:
            move_src, move_dest = move_to_index(move)
        except:
            print(move, fen_string, "\n", index, "\n", row)
            continue

        # Extract the piece at each square and append to row_data
        row_data = [board.piece_at(square).__str__() for square in chess.SQUARES]

        # Append the move index to row_data
        row_data.extend([move_src, move_dest])

        # Make the move on the board
        m = chess.Move.from_uci(move)
        board.push(m)

        # Append the row_data to the list
        rows_data.append(row_data)
    i+=1
    if i % 5000 == 0:
      print(f"{i} of {sample_count}")

# Create the result DataFrame in a single step
columns = [f"Square_{i}" for i in range(64)] + ["move_src", "move_dest"]
result_df = pd.DataFrame(rows_data, columns=columns)

In [ ]:
result_df.head(2)

# Create Model

In [ ]:
!pip install torch pandas scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming your DataFrame is named 'df'
# Extracting X and Y
X = result_df.iloc[:, :64].values
Y = result_df.iloc[:, 64:].values

# Convert 'None', 'Pawn', 'Rook', 'Knight', 'Bishop', 'Queen', 'King' to numerical values
label_encoder = LabelEncoder()
X_encoded = label_encoder.fit_transform(X.flatten()).reshape(X.shape)

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X_encoded, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_tensor, Y_tensor, test_size=0.2, random_state=42)

# Define a simple CNN model for classification
class ChessCNN(nn.Module):
    def __init__(self):
        super(ChessCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 16, 128)
        self.fc2 = nn.Linear(128, 7)  # Adjust the number of output units to match the number of classes

    def forward(self, x):
        x = x.view(-1, 1, 64)  # Reshape input for 1D convolution
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = ChessCNN()
criterion = nn.CrossEntropyLoss()  # Change the loss function to CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
batch_size = 32

train_dataset = data.TensorDataset(X_train, Y_train.long())  # Note: Use .long() for the labels
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, Y_test.long())  # Note: Use .long() for the labels
    print(f'Test Loss: {test_loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'chess_cnn_model.pth')


In [ ]:
correct_predictions = 0

with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, Y_test)

    # Count correct predictions
    for i in range(len(Y_test)):
        predicted_labels = test_outputs[i].numpy()
        true_labels = Y_test[i].numpy()
        if i == 1:
          print(predicted_labels, true_labels)
        if (predicted_labels == true_labels).all():
            correct_predictions += 1

print(f'Test Loss: {test_loss.item():.4f}')
print(f'Number of correct predictions: {correct_predictions}/{len(Y_test)}')
